In [30]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

In [2]:
# источник данных - https://www.kaggle.com/gpreda/covid-world-vaccination-progress?select=country_vaccinations.csv
df = pd.read_csv("country_vaccinations.csv")
print(df.shape)
df.head(3)

(27159, 15)


,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/


In [3]:
df.loc[df["country"] == "Russia"]

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
20343,Russia,RUS,2020-12-15,28500.0,28500.0,NaN,NaN,NaN,0.02,0.02,NaN,NaN,"EpiVacCorona, Sputnik V",Official data from local governments via gogov.ru,https://gogov.ru/articles/covid-v-stats
20344,Russia,RUS,2020-12-16,NaN,NaN,NaN,NaN,3357.0,NaN,NaN,NaN,23.0,"EpiVacCorona, Sputnik V",Official data from local governments via gogov.ru,https://gogov.ru/articles/covid-v-stats
20345,Russia,RUS,2020-12-17,NaN,NaN,NaN,NaN,3357.0,NaN,NaN,NaN,23.0,"EpiVacCorona, Sputnik V",Official data from local governments via gogov.ru,https://gogov.ru/articles/covid-v-stats
20346,Russia,RUS,2020-12-18,NaN,NaN,NaN,NaN,3357.0,NaN,NaN,NaN,23.0,"EpiVacCorona, Sputnik V",Official data from local governments via gogov.ru,https://gogov.ru/articles/covid-v-stats
20347,Russia,RUS,2020-12-19,NaN,NaN,NaN,NaN,3357.0,NaN,NaN,NaN,23.0,"EpiVacCorona, Sputnik V",Official data from local governments via gogov.ru,https://gogov.ru/articles/covid-v-stats
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20532,Russia,RUS,2021-06-22,35983165.0,20345332.0,15637833.0,745321.0,439430.0,24.66,13.94,10.72,3011.0,"EpiVacCorona, Sputnik V",Official data from local governments via gogov.ru,https://gogov.ru/articles/covid-v-stats
20533,Russia,RUS,2021-06-23,36044137.0,20390788.0,15653349.0,60972.0,403434.0,24.70,13.97,10.73,2764.0,"EpiVacCorona, Sputnik V",Official data from local governments via gogov.ru,https://gogov.ru/articles/covid-v-stats
20534,Russia,RUS,2021-06-24,36970273.0,20682043.0,16288230.0,926136.0,435312.0,25.33,14.17,11.16,2983.0,"EpiVacCorona, Sputnik V",Official data from local governments via gogov.ru,https://gogov.ru/articles/covid-v-stats
20535,Russia,RUS,2021-06-25,37603085.0,21096764.0,16506321.0,632812.0,466519.0,25.77,14.46,11.31,3197.0,"EpiVacCorona, Sputnik V",Official data from local governments via gogov.ru,https://gogov.ru/articles/covid-v-stats


In [20]:
df[["country","date", "vaccines", "daily_vaccinations_per_million", "daily_vaccinations", "people_vaccinated"]]

,country,date,vaccines,daily_vaccinations_per_million,daily_vaccinations,people_vaccinated
0,Afghanistan,2021-02-22,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",NaN,NaN,0.0
1,Afghanistan,2021-02-23,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",35.0,1367.0,NaN
2,Afghanistan,2021-02-24,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",35.0,1367.0,NaN
3,Afghanistan,2021-02-25,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",35.0,1367.0,NaN
4,Afghanistan,2021-02-26,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",35.0,1367.0,NaN
...,...,...,...,...,...,...
27154,Zimbabwe,2021-06-20,Sinopharm/Beijing,304.0,4525.0,703065.0
27155,Zimbabwe,2021-06-21,Sinopharm/Beijing,314.0,4664.0,704001.0
27156,Zimbabwe,2021-06-22,Sinopharm/Beijing,316.0,4700.0,706158.0
27157,Zimbabwe,2021-06-23,Sinopharm/Beijing,442.0,6568.0,715056.0


$\bf{Признаки:}$ "country","date", "vaccines", "daily_vaccinations_per_million", "daily_vaccinations"

$\bf{Целевая \ переменная:}$ "people_vaccinated" 

In [79]:
df.loc[df["date"] <= "2021-06-24"].loc[df["country"] == "Russia"].loc[df["daily_vaccinations_per_million"] == "NaN"][["date"]]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,date


In [76]:
#df.loc[df["date"] < "2021-06-20"].loc[df["country"] == "Russia"][["daily_vaccinations_per_million"]].mean()

for d, c in df["date"].unique(), df["country"].unique():
    df.loc[df["date"] == d].loc[df["country"] == c].loc[df["daily_vaccinations_per_million"] == NaN][["daily_vaccinations_per_million"]] = df.loc[df["date"] < d].loc[df["country"] == c][["daily_vaccinations_per_million"]].mean()

ValueError: too many values to unpack (expected 2)

In [21]:
class DataPreprocessing:

    #"country" - this is the country for which the vaccination information is provided;
    #"date" - date for the data entry; for some of the dates we have only the daily vaccinations, 
    #         for others, only the (cumulative) total;
    #"vaccines" - vaccines used in the country (up to date);
    #"daily_vaccinations_per_million" - ratio (in ppm) between vaccination number and total 
    #                                   population for the current date in the country;
    #"daily_vaccinations" - for a certain data entry, the number of vaccination for that date/country;

    def __init__(self):
        """Параметры класса"""
        self.dvperm_mean = None
        self.dv_mean = None
        
    def fit(self, X):
        """Сохранение статистик"""       
        # Расчет медиан
        self.dvperm_mean = X["daily_vaccinations_per_million"].mean()
        self.kitchen_square_quantile = X["daily_vaccinations"].mean()
    
    def transform(self, X):
        """Трансформация данных"""

        # Rooms
        X['Rooms_outlier'] = 0
        X.loc[(X['Rooms'] == 0) | (X['Rooms'] >= 6), 'Rooms_outlier'] = 1
        
        X.loc[X['Rooms'] == 0, 'Rooms'] = 1
        X.loc[X['Rooms'] >= 4, 'Rooms'] = self.medians['Rooms']
        
        # Square, LifeSquare, KitchenSquare
        
        X['Square_outlier'] = 0
        X.loc[(X['Square'] == 0) | (X['Square'] <= 8) | (X['Square'] >= 900), 'Square_outlier'] = 1
        
        condition = (X['KitchenSquare'].isna()) | (X['KitchenSquare'] > self.kitchen_square_quantile)
        
        X.loc[condition, 'KitchenSquare'] = self.medians['KitchenSquare']
        X.loc[X['KitchenSquare'] < 3 , 'KitchenSquare'] = X['Square'] 
        X.loc[X['KitchenSquare'] > 0.5*X['Square'], 'KitchenSquare'] = 0.3*X['Square'] #исключает слишком большую, относительно размеров квартиры, кухню
        
        X.loc[X['LifeSquare'] > X['Square'], 'LifeSquare'] = X['Square'] - X['KitchenSquare'] - 0.2*X['Square'] #0.2*X['Square'] на прочие удобства квартиры 
        
        # HouseFloor, Floor
        
        X['HouseFloor_outlier'] = 0
        X.loc[X['HouseFloor'] == 0, 'HouseFloor_outlier'] = 1
        X.loc[X['Floor'] > X['HouseFloor'], 'HouseFloor_outlier'] = 1
        
        X.loc[X['HouseFloor'] == 0, 'HouseFloor'] = self.medians['HouseFloor']
        
        X.loc[X['Floor'] > X['HouseFloor'], 'Floor'] = X['HouseFloor']
        
        # HouseYear

        current_year = datetime.now().year
        
        X['HouseYear_outlier'] = 0
        X.loc[X['HouseYear'] > current_year, 'HouseYear_outlier'] = 1
        
        X.loc[X['HouseYear'] > current_year, 'HouseYear'] = current_year
        
        # Healthcare_1
        if 'Healthcare_1' in X.columns:
            X.drop('Healthcare_1', axis=1, inplace=True)
        
        X.fillna(self.medians, inplace=True)
        
        return X

In [ ]:
#сначала разбить на тестовую и тренировочную

preprocessor = DataPreprocessing()
preprocessor.fit(train_df)